In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [8]:
import sys
from pathlib import Path
sys.path.append(Path(os.getcwd()).parent.parent.as_posix())

In [9]:
import dgl
import json

import optuna
import pickle

import torch
import torch.nn as nn

from functools import partial
from itertools import product
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from dataset import get_datasets, ETTDataset

from utils import seed_everything
from models.gcn import GCNModel

from constructor import construct_ess, construct_vanilla, construct_complete
from graph_features import spectral_features, deepwalk_features

from train import train_step, evaluation_step

import warnings
warnings.simplefilter("ignore")

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed_everything()

In [42]:
DATASET_NAME = "ETTh1.csv"

# LSTF setup
LOOKBACK_SIZE = 96
HORIZON_SIZE = 24

# Graphs setup
ALPHA = 0.05
GRAPH_CONSTRUCTION_FN = partial(construct_ess, alpha=ALPHA)
GRAPH_FEATURES_FN = partial(spectral_features, embed_size=21)

# Model setup
BATCH_SIZE = 32
HIDDEN_DIM = None
NUM_LAYERS = None
DROPOUT = None
ACTIVATION_FN = nn.ReLU

# Train setup
NUM_EPOCHS = 20
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
PATIENCE = 2
LR_FACTOR = 0.5

In [43]:
train_ds, val_ds, test_ds = get_datasets(
    dataset_name="weather.npy",
    lookback_size=LOOKBACK_SIZE,
    horizon_size=HORIZON_SIZE
)

In [47]:
class DatasetAdapter(Dataset):
    def __init__(self, dataset: ETTDataset, graph_construction_fn, graph_features_fn=None):
        super().__init__()
        self.graphs: list[dgl.DGLGraph] = []
        self.targets: list[torch.Tensor] = []
        for idx in tqdm(range(len(dataset)), desc="Building graphs"):
            x_data, y_data = dataset[idx]
            graph = graph_construction_fn(x_data)

            if graph_features_fn:
                graph_features = graph_features_fn(graph)
                graph.ndata["h"] = torch.cat([x_data.T, graph_features], dim=1)
            else:
                graph.ndata["h"] = x_data.T
            
            self.targets.append(y_data)
            self.graphs.append(graph)

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx) -> tuple[dgl.DGLGraph, torch.Tensor]:
        return self.graphs[idx], self.targets[idx]

In [48]:
def graph_collate_fn(batch):
    """
    Custom collate function for batching DGL graphs.
    :param graphs: batch of graphs and targets
    :returns: batched graph, batch of targets
    """
    graphs, targets = zip(*batch)
    targets_tensor = torch.stack(targets, dim=0)
    return dgl.batch(graphs), targets_tensor

In [49]:
train_adapter_ds = DatasetAdapter(
    dataset=train_ds,
    graph_construction_fn=GRAPH_CONSTRUCTION_FN,
    graph_features_fn=GRAPH_FEATURES_FN
)

val_adapter_ds = DatasetAdapter(
    dataset=val_ds,
    graph_construction_fn=GRAPH_CONSTRUCTION_FN,
    graph_features_fn=GRAPH_FEATURES_FN
)

test_adapter_ds = DatasetAdapter(
    dataset=test_ds,
    graph_construction_fn=GRAPH_CONSTRUCTION_FN,
    graph_features_fn=GRAPH_FEATURES_FN
)

Building graphs: 100%|██████████| 1043/1043 [01:12<00:00, 14.33it/s]


In [23]:
train_loader = DataLoader(
    dataset=train_adapter_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    collate_fn=graph_collate_fn
)

val_loader = DataLoader(
    dataset=val_adapter_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=graph_collate_fn
)

test_loader = DataLoader(
    dataset=test_adapter_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=graph_collate_fn
)

In [24]:
INPUT_DIM = train_adapter_ds[0][0].ndata["h"].shape[1]
INPUT_DIM

103

In [53]:
class GraphTSModel(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        num_layers: int,
        horizon_size: int,
        activation_fn: nn.Module,
        dropout: float = 0,
    ) -> "GraphTSModel":
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.activation_fn = activation_fn
        self.dropout = dropout
        self.horizon_size = horizon_size

        self.backbone = GCNModel(
            input_dim=self.input_dim,
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers,
            activation_fn=self.activation_fn,
            dropout=self.dropout
        )

        self.head = nn.Linear(self.hidden_dim, self.horizon_size)
    
    def forward(self, graph, features):
        x = features
        outputs = self.backbone(graph, x)
        tgt_emb = outputs[:21]
        outputs = self.head(tgt_emb)
        return outputs

In [66]:
next(iter(train_loader))[0].ndata["h"].shape[1]

torch.Size([672, 103])

In [55]:
model = GraphTSModel(
    input_dim=INPUT_DIM,
    hidden_dim=128,
    num_layers=1,
    horizon_size=HORIZON_SIZE,
    activation_fn=ACTIVATION_FN,
    dropout=0.1
)

model = model.to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=1e-5
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.33,
    patience=PATIENCE
)

# scheduler = None

In [56]:
setups = [
    {"horizon_size": 24, "hidden_dim": 128, "num_layers": 1, "dropout": 0.1},
    {"horizon_size": 48, "hidden_dim": 128, "num_layers": 2, "dropout": 0.1},
    {"horizon_size": 96, "hidden_dim": 128, "num_layers": 2, "dropout": 0.1},
    {"horizon_size": 168, "hidden_dim": 128, "num_layers": 2, "dropout": 0.1},
    {"horizon_size": 336, "hidden_dim": 128, "num_layers": 2, "dropout": 0.1},
]

In [75]:
results = []

for setup in setups:
    model = GraphTSModel(
        input_dim=INPUT_DIM,
        hidden_dim=128,
        num_layers=1,
        horizon_size=setup["horizon_size"],
        activation_fn=ACTIVATION_FN,
        dropout=setup["dropout"]
    )

    model = model.to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=1e-5
    )

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode="min",
        factor=0.33,
        patience=PATIENCE
    )
    
    train_loss = train_step(
        model=model,
        train_loader=train_loader,
        optimizer=optimizer,
        loss_fn=loss_fn,
        device=device
    )
    val_loss = evaluation_step(
        model=model,
        loader=val_loader,
        device=device
    )
    test_loss = evaluation_step(
        model=model,
        loader=test_loader,
        device=device
    )
        
    results[setup["horizon_size"]] = {
        "test_mse": test_loss["mse"],
        "test_mae": test_loss["mae"],   
    }
    
    if scheduler:
        scheduler.step(val_loss["mse"])

In [77]:
dict(results)

{24: {'test_mse': 0.3416346, 'test_mae': 0.3912535},
 48: {'test_mse': 0.3825235, 'test_mae': 0.4553491},
 96: {'test_mse': 0.6282355, 'test_mae': 0.5812453},
 168: {'test_mse': 0.7124853, 'test_mae': 0.6021241},
 336: {'test_mse': 0.8332592, 'test_mae': 0.7225253}}